In [35]:
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from keras.preprocessing.text import *

import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import json

General Function for getting train_test_val

In [36]:
def create_train_test_val(file):
    lst_out_text=[]
    lst_out_labels=[]
    for i in file.values():
        lst =[]
        sentence = i['text']
        sentence = sentence.lower()
        labels = i['labels']
        lst_out_text.append(sentence)
        lst_out_labels.append(labels)
    return lst_out_text,lst_out_labels    

General Function for token creation


In [37]:
def TokenCreator(sentences,val,tokenizer=None):
    
    temp_wordlist = None
    tokenizer2=None
    if(tokenizer == None):
        if val==1:
            tokenizer2 = Tokenizer(oov_token="#UK",filters='')
        else:
            tokenizer2=Tokenizer()
        tokenizer2.fit_on_texts(sentences)
        
        temp_wordlist = tokenizer2.texts_to_sequences(sentences)
        return (temp_wordlist,tokenizer2)
    
    else:
        temp_wordlist = tokenizer.texts_to_sequences(sentences)
        return temp_wordlist
    


General Function for finding unique elements

In [38]:
def find_vocab(dataset):
    lst=[]
    lst.append("#UK")
    for i in dataset:
        for j in i:
            lst.append(j)
    st = set(lst)
    return st

Reading ATE JSON Files

In [39]:
ATE_train = None
ATE_test = None
ATE_val = None
with open("Task1\processed\ATE_train.json") as f1:
    ATE_train  = json.load(f1)

with open("Task1\processed\ATE_test.json") as f2:
    ATE_test = json.load(f2)

with open("Task1\processed\ATE_val.json") as f3:
    ATE_val = json.load(f3)


Reading NER JSON Files

In [40]:
NER_train = None
NER_test = None
NER_val = None
with open("Task1\processed\\NER_train.json") as f1:
    NER_train  = json.load(f1)

with open("Task1\processed\\NER_test.json") as f2:
    NER_test = json.load(f2)

with open("Task1\processed\\NER_val.json") as f3:
    NER_val = json.load(f3)


ATE_Dataset

In [41]:
X_train_ate,Y_train_ate= create_train_test_val(ATE_train)
X_test_ate,Y_test_ate = create_train_test_val(ATE_test)
X_val_ate,Y_val_ate = create_train_test_val(ATE_val)

X_train_ate

['i charge it at night and skip taking the cord with me because of the good battery life .',
 'it is of high quality , has a killer gui , is extremely stable , is highly expandable , is bundled with lots of very good applications , is easy to use , and is absolutely gorgeous .',
 'easy to start up and does not overheat as much as other laptops .',
 'great laptop that offers many great features !',
 'one night i turned the freaking thing off after using it , the next day i turn it on , no gui , screen all dark , power light steady , hard drive light steady and not flashing as it usually does .',
 "however , the multi-touch gestures and large tracking area make having an external mouse unnecessary ( unless you 're gaming ) .",
 'i love the way the entire suite of software works together .',
 'the speed is incredible and i am more than satisfied .',
 'i can barely use any usb devices because they will not stay connected properly .',
 'when i finally had everything running with all my soft

NER_Dataset

In [42]:
X_train_ner,Y_train_ner = create_train_test_val(NER_train)
X_test_ner,Y_test_ner = create_train_test_val(NER_test)
X_val_ner,Y_val_ner = create_train_test_val(NER_val)

X_train_ner

['therefore, while interpreting statutory provisions, the courts should keep in mind the objectives or purpose for which statute has been enacted.',
 'the petitioner in w.p.no.15821 of 2008 was never considered for appointment under the national rural employment guarantee scheme either through employment exchange sponsorship or by outsourcing agencies.',
 'the factum of accident, allegation of rash and negligent driving causing death of sukendra pal singh were denied.',
 '..36..  w.a.no.655/2012 & others meaning thereby that except interview by the commission, entire procedure for recruitment as emergency appointment was followed.',
 'the law on this issue is well settled and the law is that though the provisions of evidence act are not applicable, but in a given situation the help of the principles of evidence act in the proceedings before the assessing authorities can be taken.',
 'mr. ved prakash, learned counsel appearing in writ petition no.3457/2000 submitted that the interpretat

Loading Word2Vec

In [43]:
word2vec = KeyedVectors.load_word2vec_format('WordEmbeddings/Word2Vec.bin',binary=True)

Tokenizing

In [44]:

X_train_ate_tokenized, toke = TokenCreator(X_train_ate,1)
X_test_ate_tokenized = TokenCreator(X_test_ate,0,tokenizer = toke)
X_val_ate_tokenized = TokenCreator(X_val_ate,0,tokenizer = toke)
Y_train_ate_tokenized,toke2 = TokenCreator(Y_train_ate,0)
Y_test_ate_tokenized = TokenCreator(Y_test_ate,0,tokenizer = toke2)
Y_val_ate_tokenized = TokenCreator(Y_val_ate,0,tokenizer = toke2)

print(X_val_ate_tokenized)



[[19, 2, 2026, 4, 355, 458, 33, 1, 19, 10, 1, 1, 1, 132, 31, 16, 86, 165, 306, 76, 2, 1391, 907, 383, 16, 104, 6, 4, 80, 6, 321, 4, 649, 2, 1, 5, 25, 6, 42, 22, 1, 1, 37, 3], [17, 29, 900, 161, 1, 5, 56, 110, 9, 27, 41], [621, 848, 2212, 23, 2, 784, 2303, 580, 2, 87, 53, 5, 2, 24, 1699, 266, 3], [2, 374, 14, 16, 55, 641, 207, 196, 2348, 21, 46, 10, 343, 1, 2, 325, 11, 17, 121, 3], [6, 1087, 105, 5, 48, 6, 91, 6, 43, 869, 223, 3], [6, 43, 31, 95, 356, 7, 441, 305, 2, 190, 87, 4, 7, 245, 622, 4, 5, 6, 174, 10, 183, 316, 41], [8, 114, 177, 2, 269, 757, 274, 262, 9, 71, 3], [2, 34, 54, 369, 7, 38, 22, 71, 4, 5, 26, 35, 81, 346, 13, 6, 3], [1, 21, 2, 1047, 125, 2222, 23, 8, 108, 20, 771, 19, 662, 394, 1090, 1, 3], [61, 76, 10, 287, 8, 489, 189, 6, 117, 5, 18, 453, 14, 2, 333, 35, 408, 5, 31, 28, 416, 10, 94, 333, 3], [6, 18, 223, 4, 1720, 53, 4, 5, 44, 35, 335, 166, 61, 51, 265, 141, 41], [341, 4, 2, 24, 18, 244, 463, 4, 208, 501, 57, 2, 506, 29, 4, 5, 18, 263, 1, 1, 19, 25, 3], [63, 9, 27,

In [45]:
lst=[]
for i in Y_train_ate_tokenized:
    for j in i:
        lst.append(j)
print(set(lst))

{1, 2, 3}


In [46]:
X_train_ate_tokenized

[[8, 290, 6, 49, 608, 5, 1164, 798, 2, 410, 13, 45, 70, 11, 2, 71, 34, 54, 3],
 [6,
  9,
  11,
  267,
  77,
  4,
  32,
  10,
  1165,
  799,
  4,
  9,
  411,
  1166,
  4,
  9,
  487,
  1167,
  4,
  9,
  800,
  13,
  371,
  11,
  22,
  71,
  156,
  4,
  9,
  47,
  7,
  25,
  4,
  5,
  9,
  412,
  801,
  3],
 [47, 7, 413, 53, 5, 91, 20, 1168, 39, 95, 39, 89, 164, 3],
 [27, 29, 14, 488, 131, 27, 96, 41],
 [72,
  608,
  8,
  802,
  2,
  803,
  92,
  243,
  61,
  177,
  6,
  4,
  2,
  221,
  268,
  8,
  804,
  6,
  21,
  4,
  81,
  799,
  4,
  36,
  40,
  414,
  4,
  100,
  244,
  805,
  4,
  66,
  65,
  244,
  805,
  5,
  20,
  1169,
  39,
  6,
  609,
  91,
  3],
 [328,
  4,
  2,
  415,
  1170,
  5,
  200,
  1171,
  1172,
  201,
  137,
  73,
  269,
  157,
  806,
  42,
  807,
  16,
  329,
  202,
  37,
  3],
 [8, 79, 2, 178, 2, 610, 808, 11, 82, 105, 1173, 3],
 [2, 120, 9, 1174, 5, 8, 114, 62, 57, 809, 3],
 [8, 55, 810, 25, 97, 330, 1175, 70, 28, 86, 20, 611, 811, 612, 3],
 [48,
  8,
  489,
 

Forming the vocabulary


In [47]:
st1 = find_vocab(X_train_ate_tokenized)
vocab_size_ate  = len(st1) + 1 
print(vocab_size_ate)

2406


Initializing weights

In [48]:
weights = np.zeros((vocab_size_ate, 300))

print(vocab_size_ate)
mapping = toke.word_index 
mapping

2406


{'#UK': 1,
 'the': 2,
 '.': 3,
 ',': 4,
 'and': 5,
 'it': 6,
 'to': 7,
 'i': 8,
 'is': 9,
 'a': 10,
 'of': 11,
 'for': 12,
 'with': 13,
 'that': 14,
 'my': 15,
 'you': 16,
 'this': 17,
 'was': 18,
 'in': 19,
 'not': 20,
 'on': 21,
 'very': 22,
 'but': 23,
 'computer': 24,
 'use': 25,
 'have': 26,
 'great': 27,
 'they': 28,
 'laptop': 29,
 "n't": 30,
 'so': 31,
 'has': 32,
 'are': 33,
 'battery': 34,
 'had': 35,
 'screen': 36,
 ')': 37,
 'be': 38,
 'as': 39,
 'all': 40,
 '!': 41,
 '(': 42,
 "'s": 43,
 'also': 44,
 'me': 45,
 'just': 46,
 'easy': 47,
 'when': 48,
 'at': 49,
 'or': 50,
 'only': 51,
 'which': 52,
 'up': 53,
 'life': 54,
 'can': 55,
 'windows': 56,
 'than': 57,
 'out': 58,
 'if': 59,
 'do': 60,
 'after': 61,
 'more': 62,
 'keyboard': 63,
 'would': 64,
 'drive': 65,
 'hard': 66,
 'price': 67,
 'from': 68,
 'well': 69,
 'because': 70,
 'good': 71,
 'one': 72,
 'an': 73,
 'like': 74,
 'mac': 75,
 'about': 76,
 'quality': 77,
 'its': 78,
 'love': 79,
 'get': 80,
 'no': 81,
 'so

Creating weights.

In [49]:
for word, index in mapping.items():
    if word in word2vec:
        weights[index:] = word2vec[word]

Padding the datasets.

In [50]:
X_padded_train = pad_sequences(X_train_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_train = pad_sequences(Y_train_ate_tokenized, maxlen=100, padding="pre", truncating="post")

X_padded_test = pad_sequences(X_test_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_test = pad_sequences(Y_test_ate_tokenized, maxlen=100, padding="pre", truncating="post")

X_padded_val = pad_sequences(X_val_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_val = pad_sequences(Y_val_ate_tokenized, maxlen=100, padding="pre", truncating="post")        

One-hot encoding the labels.

In [51]:
Y_padded_train = to_categorical(Y_padded_train)
Y_padded_test = to_categorical(Y_padded_test)
Y_padded_val = to_categorical(Y_padded_val)

print(Y_padded_train.shape)
print(X_padded_train.shape)


(906, 100, 4)
(906, 100)


<a href="https://www.kaggle.com/code/tanyadayanand/pos-tagging-using-rnn#1.-Preprocess-data"> Reference </a>

GRU Model

In [52]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim = vocab_size_ate,output_dim = 300,input_length  = 100,weights = [weights]))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(TimeDistributed(Dense(Y_padded_train.shape[2], activation='softmax')))

In [53]:
gru_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

gru_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 300)          721800    
                                                                 
 gru_2 (GRU)                 (None, 100, 64)           70272     
                                                                 
 time_distributed_2 (TimeDi  (None, 100, 4)            260       
 stributed)                                                      
                                                                 
Total params: 792332 (3.02 MB)
Trainable params: 792332 (3.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Training the GRU Model

In [54]:
gru_training = gru_model.fit(X_padded_train, Y_padded_train, batch_size=128, epochs=10, validation_data=(X_padded_val, Y_padded_val))

Epoch 1/10
8/8 [==============================] - 4s 202ms/step - loss: 1.2509 - acc: 0.9507 - val_loss: 0.9946 - val_acc: 0.9808
Epoch 2/10
8/8 [==============================] - 1s 102ms/step - loss: 0.7680 - acc: 0.9795 - val_loss: 0.3718 - val_acc: 0.9716
Epoch 3/10
8/8 [==============================] - 1s 96ms/step - loss: 0.2164 - acc: 0.9687 - val_loss: 0.1367 - val_acc: 0.9614
Epoch 4/10
8/8 [==============================] - 1s 98ms/step - loss: 0.1229 - acc: 0.9646 - val_loss: 0.0957 - val_acc: 0.9706
Epoch 5/10
8/8 [==============================] - 1s 106ms/step - loss: 0.0833 - acc: 0.9751 - val_loss: 0.0729 - val_acc: 0.9804
Epoch 6/10
8/8 [==============================] - 1s 103ms/step - loss: 0.0691 - acc: 0.9805 - val_loss: 0.0657 - val_acc: 0.9811
Epoch 7/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0610 - acc: 0.9808 - val_loss: 0.0584 - val_acc: 0.9812
Epoch 8/10
8/8 [==============================] - 1s 100ms/step - loss: 0.0535 - acc: 0.9814 

Testing data metrics


In [55]:
accuracy = gru_model.evaluate(X_padded_test, Y_padded_test, verbose = 1)
print(accuracy[1])

11/11 [==============================] - 0s 13ms/step - loss: 0.0524 - acc: 0.9814
0.9814329147338867
